In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# basics
import os
import time
import numpy as np

# EDA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Data preprocessing
import re
import spacy
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from concurrent.futures import ProcessPoolExecutor
from sklearn.decomposition import TruncatedSVD

# Model architecture
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import xgboost as xgb

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV


In [ ]:
# Print list of files and directories in folder
list_l = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        list_l.append(os.path.join(dirname, filename))
list_l


In [ ]:
# Set datasets
train_data = pd.read_csv(list_l[0])
test_data = pd.read_csv(list_l[1])
sample_data = pd.read_csv(list_l[2])


In [ ]:
# Cleaning
del list_l


In [ ]:
def print_short_summary(name, data):
    """
    Print data head, shape and info.
    Args:
        name (str): name of dataset
        data (dataframe): dataset in a pd.DataFrame format
    """
    print(name)
    print('\n1. Data head:')
    print(data.head())
    print('\n2 Data shape: {}'.format(data.shape))
    print('\n3. Data info:')
    data.info()
    if 'text' in data.columns:
        avg = np.mean(np.vectorize(len)(data['text']))
        print('\n4. Average number of characters per text: {:.0f}'.format(avg))


In [ ]:
print_short_summary('Train data', train_data)


In [ ]:
print_short_summary('Test data', test_data)


In [ ]:
print_short_summary('Sample data', sample_data)


In [ ]:
# Cleaning
del print_short_summary


In [ ]:
# Plot horizontal barplot of number of records per label
plt.figure(figsize=(16, 9))
tmp = train_data['author'].value_counts()
sns.barplot(y=tmp.index.values, x=tmp.values, orient='h')
plt.xlabel('Number of records')
plt.ylabel('Author')
plt.title('Number of records per author')
plt.show()


In [ ]:
# Cleaning
del tmp


In [ ]:
def plot_word_dist_author(labels, top_n_words = 10):
    """
    Plot charts with word frequencies per author
    Args:
        labels: list of authors
        top_n_words (opt): how many top words to plot in one chart
    """
    n = len(labels)
    
    # Get default seaborn's palette to match colours with the previous chart
    default_palette = sns.color_palette("deep")
    
    # Initialize subplots with 1 row and n columns
    fig, axes = plt.subplots(nrows=1, ncols=n, figsize=(16, 9))
    
    # Plot word count per author in the respective rows
    for i in range(n):
        col = i % n
        indexes = train_data['author'] == labels[i]
        w = train_data['text'][indexes].str.split(expand=True).unstack().value_counts()
        l = w[:top_n_words]/np.sum(w)*100
        axes[col].bar(l.index, l.values, color=default_palette[i])
        axes[col].set_title(labels[i])
        axes[col].set_xlabel('Words')
        axes[col].set_ylabel('Percentage of total word count (%)')

    plt.tight_layout()
    plt.show()


In [ ]:
# Plot word frequencies by author
plot_word_dist_author(['EAP', 'MWS', 'HPL'])


In [ ]:
# Cleaning
del plot_word_dist_author


In [ ]:
# Load english lemmas
spacy_process = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Compile regex pattern
pattern = re.compile(r'\b([a-zA-Z])\b|\d+|[.,!?()-:;]')

# Set english stopwords such as "the", "of", "and", "to" etc.
stop_words = set(stopwords.words('english'))


In [ ]:
def get_processed_text(text):
    """
    Return lemmatized text without single letters and digits.
    Everything is in the lower case register.
    Args:
        text (str): text of an article
    Returns:
        text (str): cleand text
    """
    # Convert to lowercase and remove digits, single letters
    text = pattern.sub('', text.lower())
    
    # Get lemma tokens
    lemmas = spacy_process(text)
    lemmas = [token.lemma_ for token in lemmas if token.text not in stop_words]

    # Join the words back into a string
    text = ' '.join(lemmas)
    
    return text

def get_clean_text(texts):
    """
    Return cleaned text.
    Execution in parallel.
    
    Args:
        texts: numpy array of string elements
    Returns:
        clean_texts: numpy array of cleaned string elements
    """
    # Set up parallel tasks processing 
    with ProcessPoolExecutor() as executor:
        clean_texts = list(executor.map(get_processed_text, texts))
        
    return np.array(clean_texts)


In [ ]:
# Get cleaned train and test texts
train_clean_data = get_clean_text(train_data['text'].values)
test_clean_data = get_clean_text(test_data['text'].values)


In [ ]:
# Print an example of the cleaned text
train_clean_data[0]


In [ ]:
# Cleaning
del spacy_process, pattern, stop_words
del get_clean_text, get_processed_text


In [ ]:
# Create vectorizer to convert plain text into feature matrix
vectorizer = TfidfVectorizer(sublinear_tf = True
                             , ngram_range = (1,2)
                             )
tfidf_vect = vectorizer.fit(train_clean_data)


In [ ]:
# Set training and testing data for the upcoming models
X_train_tfidf = tfidf_vect.transform(train_clean_data)
X_test_tfidf = tfidf_vect.transform(test_clean_data)


In [ ]:
del vectorizer, test_data, train_clean_data, test_clean_data


In [ ]:
# Reduce dimensionality of TF-IDF matrices
svd = TruncatedSVD(n_components=100)
X_train_svd = svd.fit_transform(X_train_tfidf)
X_test_svd = svd.transform(X_test_tfidf)


In [ ]:
# Standardize features without centering
scaler = StandardScaler(with_mean = False)
X_train_stand = scaler.fit_transform(X_train_tfidf)
X_test_stand = scaler.fit_transform(X_test_tfidf)

X_train_stand_svd = scaler.fit_transform(X_train_svd)
X_test_stand_svd = scaler.fit_transform(X_test_svd)


In [ ]:
# Cleaning
del scaler


In [ ]:
# Encode labels using OneHotEncoder
dict_map = {'EAP':0, 'MWS':1, 'HPL':2,}
y_train = train_data['author'].map(dict_map).values


In [ ]:
# Cleaning
del label_encoder, train_data


In [ ]:
# Set config for GridSearchCV
config_logreg = {
    'model': LogisticRegression()
    , 'name': 'Log Reg'
    , 'param_grid':
    {
        'class_weight': ['balanced']
        , 'C': [0.5, 1.0, 1.5]
        , 'solver': ['saga', 'lbfgs']
    }
}


In [ ]:
# Set config for GridSearchCV
config_rf = {
    'model': RandomForestClassifier()
    , 'name': 'Random Forest'
    , 'param_grid':
    {
        'n_estimators': [50, 75, 100]
        , 'max_depth': [10, 20]
    }
}


In [ ]:
# Set config for GridSearchCV
config_nb = {
    'model': MultinomialNB()
    , 'name': 'Mult NB'
    , 'param_grid':
    {
        'alpha': [0.001, 0.01, 0.1, 0.2, 0.4]
    }
}


In [ ]:
# Set config for GridSearchCV
config_svm = {
    'model': SVC()
    , 'name': 'SVM'
    , 'param_grid':
    {
        'class_weight': ['balanced']
        , 'probability': [True]
        , 'max_iter': [100]
        , 'C': [0.5, 1.0, 1.5]
        , 'kernel': ['rbf', 'sigmoid']
    }
}


In [ ]:
# Set config for GridSearchCV
config_xgb = {
    'model': xgb.XGBClassifier()
    , 'name': 'XGBoost'
    , 'param_grid':
    {
        'objective': ['multi:softmax']
        , 'num_class': [3]
        , 'eval_metric': ['mlogloss']
        , 'n_estimators': [100]
        , 'eta': [0.2, 0.3, 0.4]
        , 'max_depth': [5, 7]
    }
}


In [ ]:
def get_grid(config_model, X_train, y_train):
    """
    Return grid of GridSearchCV results from selected models and their parameters.
    Execution in parallel.
    
    Args:
        config_model (dict): dictionary of model's parameters
        X_train (ndarray): data to train
        y_train (ndarray): data labels
    Returns:
        dict: a dictionary with the results from training via GridSeachCV
    """
    # GridSearchCV utilizes all available CPU cores with n_jobs = -1.
    grid = GridSearchCV(config_model['model']
                        , config_model['param_grid']
                        , return_train_score = True
                        , scoring = 'neg_log_loss'
                        , cv = 5
                        , n_jobs = -1)

    # Fit object to training data
    grid = grid.fit(X_train, y_train)
        

    return grid


def get_model_results(model_name, grid_results, data_standardized):
    """
    Return a dictionary of model results.
    
    Args:
        model_name (str): name of a model
        grid_results (dict): GridSearchCV dictionary of model results
        data_standardized (1 or 0): data is stadardized
    Returns:
        dict: dictionary with updated columns
    """
    runtime = grid_results['mean_fit_time'] + grid_results['mean_score_time']
    model_results = {
        'model': model_name
        , 'params': grid_results['params']
        , 'data_std': data_standardized
        , 'mean_runtime (sec)': runtime
        , 'mean_train_score (logloss)': -grid_results['mean_train_score']
        , 'mean_test_score (logloss)': -grid_results['mean_test_score']
    }
    
    return model_results

def get_table_results_sorted(list_dict):
    """
    Convert list of dictionaries and sort by mean_test_score and mean_runtime.
    
    Args:
        list_dict (list): list of dictionaries from GridSearchCV
    Returns:
        DataFrame: sorted dataframe by mean_test_score and mean_runtime
    """
    table = [pd.DataFrame(results) for results in list_dict]   
    table = pd.concat(table)
    table = table.sort_values(by = ['mean_test_score (logloss)'
                                  ,'mean_runtime (sec)']
                        , ascending = [True, True])
    
    return table

def get_final_table_results(config_models, config_data):
    """
    Return table with every trained model and its results from GridSearchCV.
    
    Args:
        config_models (list): list of model configurations
        confg_data (list): list of data configurations
    Returns:
        DataFrame: sorted dataframe by mean_test_score and mean_runtime
    """
    n = len(config_models)
    m = len(config_data)
    table = []
    for i in range(n):
        for j in range(m):
            print('model: {} of {} | data: {} of {}'.format(i+1,n,j+1,m))
            grid = get_grid(config_models[i], config_data[j], y_train)
            model_results = get_model_results(config_models[i]['name']
                                              , grid.cv_results_
                                             , j)
            table.append(model_results)
    
    table = get_table_results_sorted(table)

    return table


In [ ]:
# Create table of Multinomial Naive Bayes results
table_nb = get_final_table_results([config_nb], [X_train_tfidf, X_train_stand])


In [ ]:
# Set list of models except Mult NB and non/standardized X_train
config_models = [config_logreg, config_rf, config_svm, config_xgb]
config_data = [X_train_svd, X_train_stand_svd]


In [ ]:
# Get table of every model except Mult NB
table_results = get_final_table_results(config_models, config_data)


In [ ]:
# Concatenate tables and print final rankings
table_results = get_table_results_sorted([table_results, table_nb])
table_results


In [ ]:
# Cleaning
del config_models, config_data, table_results
del get_results_table, get_grid, get_final_table_results


In [ ]:
# Fit top performance model
model = MultinomialNB(alpha = 0.01).fit(X_train_tfidf, y_train)


In [ ]:
# Get predicted probabilities of classes
results = model.predict_proba(X_test_tfidf)


In [ ]:
# Create submission talbe
table_submis = pd.DataFrame(results, columns=['EAP', 'MWS', 'HPL'])
table_submis['id'] = sample_data['id']


In [ ]:
# Print submission table
table_submis


In [ ]:
# Make submission
table_submis.to_csv('submission.csv', index=False)


In [ ]:
# Cleaning
del sample_data
